## Exampville Destination Choice

In [1]:
import os
import numpy as np
import pandas as pd
import larch
import larch.exampville
from larch import P, X

In [2]:
hh = pd.read_csv( larch.exampville.files.hh )
pp = pd.read_csv( larch.exampville.files.person )
tour = pd.read_csv( larch.exampville.files.tour )
skims = larch.OMX( larch.exampville.files.skims, mode='r' )

In [4]:
emp = pd.read_csv(larch.exampville.files.employment, index_col='TAZ')
emp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 1 to 40
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   NONRETAIL_EMP  40 non-null     int64
 1   RETAIL_EMP     40 non-null     int64
 2   TOTAL_EMP      40 non-null     int64
dtypes: int64(3)
memory usage: 1.2 KB


In [11]:
logsums = pd.read_pickle('./logsums.pkl.gz')
logsums.shape

(7564, 40)

In [9]:
co = tour.merge(hh, on='HHID').merge(pp, on=('HHID','PERSONID'))
co["HOMETAZi"] = co["HOMETAZ"] - 1
co["DTAZi"] = co["DTAZ"] - 1
co = co[co.TOURPURP == 1]
co.index.name = 'CASE_ID'
co.shape

(7564, 35)

In [10]:
co.head()

,TOURID,HHID,PERSONID,DTAZ,TOURMODE,TOURPURP,N_STOPS,N_TRIPS_x,N_TRIPS_HBW_x,N_TRIPS_HBO_x,...,WORKS,N_WORK_TOURS,N_OTHER_TOURS,N_TOURS,N_TRIPS,N_TRIPS_HBW,N_TRIPS_HBO,N_TRIPS_NHB,HOMETAZi,DTAZi
CASE_ID,,,,,,,,,,,,,,,,,,,,,
0,0,50000,60000,22,1,1,0,2,2,0,...,1,1,0,1,2,2,0,0,21,21
1,1,50000,60001,4,1,1,0,2,2,0,...,1,1,1,2,4,2,2,0,21,3
3,3,50000,60002,20,1,1,0,2,2,0,...,1,1,1,2,5,2,2,1,21,19
7,7,50001,60004,25,1,1,2,4,1,1,...,1,1,0,1,4,1,1,2,21,24
10,10,50003,60006,25,1,1,1,3,1,1,...,1,1,2,3,12,1,5,6,21,24


In [16]:
skims.AUTO_DIST[:][co["HOMETAZi"],:].shape

(7564, 40)

In [17]:
distance = pd.DataFrame(skims.AUTO_DIST[:][co["HOMETAZi"],:],
                        index=co.index, columns=skims.TAZ_ID)

In [18]:
distance.shape

(7564, 40)

In [19]:
ca = pd.concat([distance.stack().rename("distance"), 
                logsums.stack().rename("logsum")], axis=1)
ca.shape

(302560, 2)

In [21]:
ca.head()

distance    logsum
CASE_ID TAZ_ID                    
0       1       8.286090 -2.935611
        2       8.256698 -2.660436
        3       2.928829 -1.745654
        4       5.502758 -2.229281
        5       5.882862 -2.403117

In [22]:
emp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 1 to 40
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   NONRETAIL_EMP  40 non-null     int64
 1   RETAIL_EMP     40 non-null     int64
 2   TOTAL_EMP      40 non-null     int64
dtypes: int64(3)
memory usage: 1.2 KB


In [23]:
emp.head()

,NONRETAIL_EMP,RETAIL_EMP,TOTAL_EMP
TAZ,,,
1,118,340,458
2,91,0,91
3,197,2,199
4,277,8,285
5,44,13,57


In [24]:
ca = ca.join(emp, on="TAZ_ID")
ca.shape

(302560, 5)

In [25]:
area_type = pd.Series(
    skims.TAZ_AREA_TYPE[:],
    index=skims.TAZ_ID[:],
    name ='TAZ_AREA_TYPE',
).astype('category')

In [26]:
ca = ca.join(area_type, on='TAZ_ID')

In [28]:
co.shape, ca.shape

((7564, 35), (302560, 6))

In [32]:
co.head()

,TOURID,HHID,PERSONID,DTAZ,TOURMODE,TOURPURP,N_STOPS,N_TRIPS_x,N_TRIPS_HBW_x,N_TRIPS_HBO_x,...,WORKS,N_WORK_TOURS,N_OTHER_TOURS,N_TOURS,N_TRIPS,N_TRIPS_HBW,N_TRIPS_HBO,N_TRIPS_NHB,HOMETAZi,DTAZi
CASE_ID,,,,,,,,,,,,,,,,,,,,,
0,0,50000,60000,22,1,1,0,2,2,0,...,1,1,0,1,2,2,0,0,21,21
1,1,50000,60001,4,1,1,0,2,2,0,...,1,1,1,2,4,2,2,0,21,3
3,3,50000,60002,20,1,1,0,2,2,0,...,1,1,1,2,5,2,2,1,21,19
7,7,50001,60004,25,1,1,2,4,1,1,...,1,1,0,1,4,1,1,2,21,24
10,10,50003,60006,25,1,1,1,3,1,1,...,1,1,2,3,12,1,5,6,21,24


In [33]:
ca.head()

distance    logsum  NONRETAIL_EMP  RETAIL_EMP  TOTAL_EMP  \
CASE_ID TAZ_ID                                                             
0       1       8.286090 -2.935611            118         340        458   
        2       8.256698 -2.660436             91           0         91   
        3       2.928829 -1.745654            197           2        199   
        4       5.502758 -2.229281            277           8        285   
        5       5.882862 -2.403117             44          13         57   

               TAZ_AREA_TYPE  
CASE_ID TAZ_ID                
0       1             b'SUB'  
        2             b'SUB'  
        3             b'URB'  
        4             b'URB'  
        5             b'SUB'

In [43]:
skims.TAZ_ID

/lookup/TAZ_ID (CArray(40,), shuffle, zlib(1)) ''
  atom := Int64Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (8192,)

In [27]:
dfs = larch.DataFrames(
    co=co,
    ca=ca,
    alt_codes=skims.TAZ_ID, 
    alt_names=['TAZ{i}' for i in skims.TAZ_ID],
    ch_name='DTAZ',
    av=True,
)

## Model Definition

In [34]:
m = larch.Model(dataservice=dfs)

In [35]:
m.quantity_ca = (
        + P.EmpRetail_HighInc * X('RETAIL_EMP * (INCOME>50000)')
        + P.EmpNonRetail_HighInc * X('NONRETAIL_EMP') * X("INCOME>50000")
        + P.EmpRetail_LowInc * X('RETAIL_EMP') * X("INCOME<=50000")
        + P.EmpNonRetail_LowInc * X('NONRETAIL_EMP') * X("INCOME<=50000")
)

In [36]:
m.quantity_scale = P.Theta

In [37]:
m.utility_ca = (
    + P.logsum * X.logsum
    + P.distance * X.distance
)

In [38]:
m.load_data()

req_data does not request {choice_ca,choice_co,choice_co_code} but choice is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided


In [39]:
m.maximize_loglike()

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
EmpNonRetail_HighInc,0.681995,0.0,0.0,-inf,inf,0,,0.681995
EmpNonRetail_LowInc,-0.440679,0.0,0.0,-inf,inf,0,,-0.440679
EmpRetail_HighInc,-0.681995,0.0,0.0,-inf,inf,0,,-0.681995
EmpRetail_LowInc,0.440679,0.0,0.0,-inf,inf,0,,0.440679
Theta,0.749372,1.0,1.0,0.001,1.0,0,,0.749372
distance,-0.041823,0.0,0.0,-inf,inf,0,,-0.041823
logsum,1.020814,0.0,0.0,-inf,inf,0,,1.020814


/home/danph/.pyenv/versions/miniconda3-4.7.10/envs/.dcm/lib/python3.9/site-packages/larch/model/optimization.py:306: UserWarning: slsqp may not play nicely with unbounded parameters
if you get poor results, consider setting global bounds with model.set_cap()
  warnings.warn( # infinite bounds # )


┣                x: EmpNonRetail_HighInc    0.681995
┃                   EmpNonRetail_LowInc    -0.440679
┃                   EmpRetail_HighInc      -0.681995
┃                   EmpRetail_LowInc        0.440679
┃                   Theta                   0.749372
┃                   distance               -0.041823
┃                   logsum                  1.020814
┃                   dtype: float64
┣          loglike: -25157.72676051146
┣        d_loglike: EmpNonRetail_HighInc    0.000315
┃                   EmpNonRetail_LowInc    -0.000012
┃                   EmpRetail_HighInc      -0.000315
┃                   EmpRetail_LowInc        0.000012
┃                   Theta                   0.007939
┃                   distance                0.004064
┃                   logsum                 -0.000868
┃                   dtype: float64
┣              nit: 13
┣             nfev: 31
┣             njev: 13
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(seconds=1, microseconds=963544)
┣           method: 'slsqp'
┣          n_cases: 7564
┣ iteration_number: 13
┣          logloss: 3.3259818562283794

In [40]:
m.calculate_parameter_covariance()

<ipython-input-40-69910d0b056e>:1: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  m.calculate_parameter_covariance()
/home/danph/.pyenv/versions/miniconda3-4.7.10/envs/.dcm/lib/python3.9/site-packages/larch/linalg/__init__.py:18: UserWarning: minimum eig 4.976990965132953e-06 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")


In [41]:
m.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value
EmpNonRetail_HighInc,0.682,274.,0.00,,0.00
EmpNonRetail_LowInc,-0.441,317.,-0.00,,0.00
EmpRetail_HighInc,-0.682,274.,-0.00,,0.00
EmpRetail_LowInc,0.441,317.,0.00,,0.00
Theta,0.749,0.0152,-16.45,***,1.00
distance,-0.0418,0.0107,-3.90,***,0.00
logsum,1.02,0.0317,32.16,***,0.00


In [42]:
m.utility_functions()

+ P.logsum * X.logsum + P.distance * X.distance + P.Theta * log( + exp(P.EmpRetail_HighInc) * X('RETAIL_EMP * (INCOME>50000)') + exp(P.EmpNonRetail_HighInc) * X('NONRETAIL_EMP*(INCOME>50000)') + exp(P.EmpRetail_LowInc) * X('RETAIL_EMP*(INCOME<=50000)') + exp(P.EmpNonRetail_LowInc) * X('NONRETAIL_EMP*(INCOME<=50000)'))
